# Part **B**
1. Write programs to developed models using different version of decision tree.
2. Apply 10-fold cross validation
3. Store your results in terms of accuracy and F-measure with single excel file with multiple rows


In [ ]:
from random import seed
from random import randrange
from csv import reader

def load_csv(fName):
    file=open(fName,"r")
    lines=reader(file)
    dataset=list(lines)
    return dataset

def str_column_to_float(dataSet, column):
   for row in dataSet:
       row[column] = float(row[column].strip())

def cross_validation_split(dataset,folds):
   dataset_split = list()
   dataset_copy = list(dataset)
   fold_size = int(len(dataset) / folds)
   for i in range(folds):
       fold = list()
       while len(fold) < fold_size:
           index = randrange(len(dataset_copy))
           fold.append(dataset_copy.pop(index))
       dataset_split.append(fold)
   return dataset_split

def accuracy_metric(actual, predicted):
   correct = 0
   for i in range(len(actual)):
       if actual[i] == predicted[i]:
           correct += 1
   return correct / float(len(actual)) * 100.0

def evaluate_algorithm(dataset, algorithm, fold_cnt, *args):
   folds = cross_validation_split(dataset, fold_cnt)
   scores = list()
   for fold in folds:
       train_set = list(folds)
       train_set.remove(fold)
       train_set = sum(train_set, [])
       test_set = list()
       for row in fold:
           row_copy = list(row)
           test_set.append(row_copy)
           row_copy[-1] = None
       predicted = algorithm(train_set, test_set, *args)
       actual = [row[-1] for row in fold]
       accuracy = accuracy_metric(actual, predicted)
       scores.append(accuracy)
   return scores

def test_split(index, value, dataSet):
   left, right = list(), list()
   for row in dataSet:
       if row[index] < value:
           left.append(row)
       else:
           right.append(row)
   return left, right

def gini_index(groups, classes):
   n_instances = float(sum([len(group) for group in groups]))
   gini = 0.0
   for group in groups:
       size = float(len(group))
       if size == 0:
           continue
       score = 0.0
       for class_val in classes:
           p = [row[-1] for row in group].count(class_val) / size
           score += p * p
       gini += (1.0 - score) * (size / n_instances)
   return gini

def get_split(dataSet):
   class_values = list(set(row[-1] for row in dataSet))
   b_index, b_value, b_score, b_groups = 999, 999, 999, None
   for index in range(len(dataSet[0])-1):
       for row in dataSet:
           groups = test_split(index, row[index], dataSet)
           gini = gini_index(groups, class_values)
           if gini < b_score:
               b_index, b_value, b_score, b_groups = index, row[index], gini, groups
   return {'index':b_index, 'value':b_value, 'groups':b_groups}

def to_terminal(grp):
   outcomes = [row[-1] for row in grp]
   return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
   left, right = node['groups']
   del(node['groups'])
   if not left or not right:
       node['left'] = node['right'] = to_terminal(left + right)
       return
   if depth >= max_depth:
       node['left'], node['right'] = to_terminal(left), to_terminal(right)
       return
   if len(left) <= min_size:
       node['left'] = to_terminal(left)
   else:
       node['left'] = get_split(left)
       split(node['left'], max_depth, min_size, depth+1)
   if len(right) <= min_size:
       node['right'] = to_terminal(right)
   else:
       node['right'] = get_split(right)
       split(node['right'], max_depth, min_size, depth+1)

def build_tree(train, max_depth, min_size):
   root = get_split(train)
   split(root, max_depth, min_size, 1)
   return root

def predict(node, row):
   if row[node['index']] < node['value']:
       if isinstance(node['left'], dict):
           return predict(node['left'], row)
       else:
           return node['left']
   else:
       if isinstance(node['right'], dict):
           return predict(node['right'], row)
       else:
           return node['right']

def decision_tree(train, test, max_depth, min_size):
   tree = build_tree(train, max_depth, min_size)
   predictions = list()
   for row in test:
       prediction = predict(tree, row)
       predictions.append(prediction)
   return(predictions)

seed(1)
for i in range(1,2):
    filename = str(i)+'.csv'
    dataset = load_csv(filename)
    for i in range(len(dataset[0])):
        str_column_to_float(dataset, i)
        n_folds = 10
        max_depth = 5
        min_size = 10
        scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
        print('Scores: %s' % scores)
        print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

In [ ]:
def normalizedata(X_train):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    return X_train

In [ ]:
def model1(trdata,tract,tsdata):
    model = DecisionTreeClassifier().fit(trdata,tract)
    pred= model.predict(tsdata)
    return pred

def model2(trdata,tract,tsdata):
    model = DecisionTreeClassifier(criterion='entropy').fit(trdata,tract)
    pred= model.predict(tsdata)
    return pred

def modeln1(trdata,tract,tsdata):
    model = MultinomialNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    return pred

def modeln2(trdata,tract,tsdata):
    model = BernoulliNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    return pred

def modeln3(trdata,tract,tsdata):
    model = GaussianNB()
    model.fit(trdata,tract)
    pred= model.predict(tsdata)
    return pred

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(10)

In [ ]:
for i in range(1,2):
    print(i)
    fname=str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',')
    data[:,0:-1]=normalizedata(data[:,0:-1])
    in1=np.where(data[:,-1]>0)
    data[in1[0],-1]=1
    predvalue=np.zeros((np.shape(data)[0],6))
    for train_index, test_index in kf.split(data):
        trdata=data[train_index,0:-1]
        tsdata=data[test_index,0:-1]
        tract=data[train_index,-1]
        tsact=data[test_index,-1]
        predvalue[test_index,0]=model1(trdata,tract,tsdata)
        predvalue[test_index,1]=model2(trdata,tract,tsdata)
        predvalue[test_index,2]=modeln1(trdata,tract,tsdata)
        predvalue[test_index,3]=modeln3(trdata,tract,tsdata)
        predvalue[test_index,4]=modeln3(trdata,tract,tsdata)
        predvalue[test_index,5]=tsact
    print(predvalue)
    fname=str(i)+'asnswer.csv'    
    np.savetxt(fname,predvalue, delimiter=',', fmt='%f') 

In [ ]:
from sklearn.metrics import (f1_score, precision_score, recall_score,accuracy_score)

fval=np.zeros((56,5))
acv=np.zeros((56,5))
for i in range(0,56):
    print(i+1)
    fname= str(i+1)+'.csv'
    data1=np.genfromtxt(fname,delimiter=',')
    y1=data1[:,-1]
    for j in range(0,5):
        fval[i,j]=f1_score(y1, data1[:,j], average = 'weighted')
        acv[i,j]=accuracy_score(y1, data1[:,j])   
    print(fval, acv)


fname='acc.csv'    
np.savetxt(fname,acv, delimiter=',', fmt='%f')      
fname='fmea.csv'
np.savetxt(fname,fval, delimiter=',', fmt='%f')  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('fmea.csv', header = None)
print(df.head(3))
df.columns = ['Model1', 'Model2', 'Model3', 'Model4', 'Model5']
print(df.head(3))

sns.boxplot(data=df,palette="colorblind")
plt.savefig('F-measure.png')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('acc.csv', header = None)
print(df.head(3))
df.columns = ['Model1', 'Model2', 'Model3', 'Model4', 'Model5']
print(df.head(3))

sns.boxplot(data=df,palette="colorblind")
plt.savefig('accuracy.png')